#### 사용자 정의 함수

---
- 함수기능 : 모델 학습 진행 함수
- 함수이름 : training
- 매개변수 : 함수 구동 시 필요한 재료 => 학습을 위한 재료
    * 모델 인스턴스
    * 학습 데이터셋 => 피쳐와 타겟 (학습전 Tensor화)
    * 손실함수 인스턴스
    * 최적화 인스턴스
    * 학습횟수 => 에포크 EPOCH
    * 배치크기 => BATCH_SIZE
    * 배치개수 => BATCH_CNT
    * 검증용 데이터셋 => 피쳐와 타겟 (검증전 Tensor화)
- 함수결과 : 학습 시 에포크당 손실값과 성능지표값, 검증의 손실값과 성능지표값
---

---
- 함수기능 : 에포크 단위 모델 학습 진행 함수
- 함수이름 : epochTraining
- 매개변수 : 함수 구동 시 필요한 재료 => 학습을 위한 재료
    * 모델 인스턴스
    * 학습용 데이터셋 => 피쳐와 라벨 (학습전 Tensor화)
    * 손실함수 인스턴스
    * 최적화 인스턴스
    * 배치크기 => BATCH_SIZE
    * 배치개수 => BATCH_CNT
- 함수결과 : 손실값과 성능지표값
---

In [17]:
# 모듈로딩
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchmetrics.regression import R2Score
from torchmetrics.classification import F1Score

# 텐서 저장 및 실행 위치 설정
DEVICE='cuda' if torch.cuda.is_available() else 'cpu'

In [18]:
# 검증 및 테스트 진행 후 손실값과 성능지표값 반환 함수
def testing(model, feature, target, lossFunc, scoreFunc):
    
    # - 최적화 기능 비활성화 후 데이터셋 추출 후 학습 진행
    with torch.no_grad():
        # 데이터셋 텐서화
        featureTS=torch.FloatTensor(feature.values).to(DEVICE)
        targetTS=torch.FloatTensor(target.values).to(DEVICE)

        # 학습 진행
        pre_y=model(featureTS).to(DEVICE)

        # 손실 계산
        loss=lossFunc(pre_y, targetTS)

        # 점수 추출
        score=scoreFunc(pre_y, targetTS)

    # 손실값과 성능지표값 반환
    return loss, score

In [19]:
# 에포크 당 학습 진행 후 손실값과 성능지표값 반환 함수
def epochTraining(model, feature, target, lossFunc, optimizer, batch_cnt, batch_size=32, is_class=True):
    # - 에포크에서 배치크기만큼 데이터셋 추출 후 학습 진행
    loss_total, score_total=0, 0
    for batch in range(batch_cnt):
        # 배치크기만큼 데이터셋 추출 및 텐서화
        start=batch*batch_size # [예 batch_size=32 => 0, 32, 64,...]
        end=start + batch_size # [예 batch_size=32 => 32, 64, 96, ...]

        x_train=torch.FloatTensor(feature[start:end].values).to(DEVICE)
        y_train=torch.FloatTensor(target[start:end].values).to(DEVICE)

        # 학습 진행
        pre_y=model(x_train).to(DEVICE)

        # 손실 계산
        loss=lossFunc(pre_y, y_train)
        loss_total += loss

        # 점수 추출
        score=R2Score()(pre_y, y_train) if is_class else R2Score()(pre_y, y_train)
        score_total += score

        # 최적화 : W,b를 업데이트
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # 에포크당 손실값과 성능지표값 반환
    # 테스트 및 검증 함수 사용
    return loss_total/batch_cnt, score_total/batch_cnt

In [20]:
# 학습 진행
EPOCHS=10
TV_LOSS={'TRAIN':[], 'VAL':[]}
TV_SCORE={'TRAIN':[], 'VAL':[]}

for epoch in range(EPOCHS):
    # 학습 진행
    train_loss, train_score=epochTraining()
    # 검증 진행
    val_loss, val_score = testing()

    # 에포크당 학습 및 검증 결과 저장
    TV_LOSS['TRAIN'].append(train_loss)
    TV_SCORE['TRAIN'].append(train_score)
    TV_LOSS['VAL'].append(val_loss)
    TV_SCORE['VAL'].append(val_score)

    # 에포크당 학습 및 검증 결과 출력
    print(f'[{epoch}.{EPOCHS}]\n- [TRAIN] LOSS : {train_loss} SCORE : {train_score}')
    print(f'[{epoch}.{EPOCHS}]\n- [VAL] LOSS : {val_loss} SCORE : {val_score}')

TypeError: epochTraining() missing 6 required positional arguments: 'model', 'feature', 'target', 'lossFunc', 'optimizer', and 'batch_cnt'